# WEEK 3: GCP Service

## What is GCP Services
Google Cloud Platform (GCP) services are cloud-based tools and APIs that let applications automatically perform tasks like uploading files, processing data, accessing storage, or interacting with other Google services such as Gmail, Drive, and BigQuery.

## Google Drive

Task: 
- download
- upload
- make new folder

## Google Sheets

Task: 
- read data
- add new column
- add new row
- add new sheet
- update row

## Google Mail

Task: 
- read email
- download attachment file
- send email

## 